In [109]:
import pandas as pd
import numpy as np
import pickle

from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.python.keras.layers import Dense, Dropout, Input
from tensorflow.python.keras.models import Model

In [1]:
infile = open('met_total_data','rb')
met_data = pickle.load(infile)
infile.close()

In [2]:
met_data.head()

,id,type1,type2,type3,color1,color2,color3
0,189744,column,structure,iron,silver,darkslategray,darkslategray
1,544740,stone carving,sculpture,classical sculpture,lightsteelblue,darkgray,black
2,191733,black and white,still life photography,drinkware,gray,dimgray,lightslategray
3,27362,weapon,sketch,drawing,gray,dimgray,darkgray
4,27362,weapon,sketch,drawing,gray,dimgray,darkgray


In [4]:
met_images = np.load('met_total_images.npy', allow_pickle=True)

In [38]:
met_images[100].shape

(3879, 2582, 3)

In [42]:
met_images.shape

(630,)

In [125]:
# cropping images from the center in equal sizes

processed_met_images = []
remove_list = []

for j, i in enumerate(met_images): 
    if i.shape[0] < 512 or i.shape[1] < 512:
        remove_list.append(j)
#         temp = np.pad(i,pad_width=((0,0),(0,0)))
#         processed_met_images.append(\
#                     temp[int(temp.shape[0]/2) - 256:int(temp.shape[0]/2) + 256,\
#                     int(i.shape[1]/2) - 256:int(temp.shape[1]/2) + 256])

    else:
        processed_met_images.append(\
                    i[int(i.shape[0]/2) - 256:int(i.shape[0]/2) + 256,\
                    int(i.shape[1]/2) - 256:int(i.shape[1]/2) + 256])

In [127]:
met_data_n = met_data.drop(remove_list, axis=0)

In [128]:
len(met_data_n), len(processed_met_images)

(589, 589)

In [129]:
img = Image.fromarray(processed_met_images[15], 'RGB')
img.show()

In [52]:
processed_met_images[5].shape

(512, 512, 3)

In [130]:
input_tensor = Input(shape=(512,512,3))
model = VGG16(weights='imagenet', include_top=False, input_tensor = input_tensor)

In [133]:
X_train = processed_met_images[:450]
y_train = met_data_n['type1'][:450]

X_test = processed_met_images[450:]
y_test = met_data_n['type1'][450:]

In [134]:
len(X_train), len(y_train), len(X_test), len(y_test)

(450, 450, 139, 139)

In [135]:
len(y_train.unique())

69

In [136]:
# 모델 Layer 데이터화
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# Layer 추가
x = layer_dict['block5_pool'].output
# Cov2D Layer +
x = Conv2D(filters = 64, kernel_size=(3, 3), activation='relu')(x)
# MaxPooling2D Layer +
x = MaxPooling2D(pool_size=(2, 2))(x)
# Flatten Layer +
x = Flatten()(x)
# FC Layer +
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(69, activation='softmax')(x)

# new model 정의
new_model = Model(inputs = model.input, outputs = x)

In [137]:
for layer in new_model.layers[:19] :
    layer.trainable = False

new_model.summary()

# 컴파일 옵션
new_model.compile(loss='sparse_categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 512, 512, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 512, 512, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 256, 256, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 256, 256, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 256, 256, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 128, 128)     0   

In [153]:
from sklearn.preprocessing import LabelEncoder

label_encode = LabelEncoder()
label_encode2 = LabelEncoder()
y_train_encoded = label_encode.fit_transform(y_train)
y_test_encoded = label_encode2.fit_transform(y_test)


In [154]:
type1_labelmap_train = dict(zip(label_encode.classes_, \
                          label_encode.transform(label_encode.classes_)))
type1_labelmap_test = dict(zip(label_encode2.classes_, \
                          label_encode2.transform(label_encode2.classes_)))

In [160]:
len(X_train), len(y_train)

(450, 450)

In [159]:
X_train_np = np.array(X_train)
X_test_np = np.array(X_test)

/var/folders/g5/grw4k9x12rbf2r7lsfh6pc480000gn/T/ipykernel_10002/800520715.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_np = np.array(X_train)


ValueError: could not broadcast input array from shape (512,512,3) into shape (512,512)

In [155]:
history = new_model.fit(X_train, y_train_encoded, epochs=5, \
                        validation_data=(X_test, y_test_encoded))

ValueError: Data cardinality is ambiguous:
  x sizes: 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 5